First, load the 311 complaints dataset available here https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

In [222]:
##https://www.zenrows.com/blog/pyppeteer#use

#how to make best magic english notebook ever
#write 5 noetbooks -> make code super clean and well documented -> use cursor -> relearn emacs 
#make it so people can easily contribute by reading notebook and adding their own function


In [62]:
# pd.options.display.max_rows = 2000

# counts = pdf['Category'].value_counts()
# print(counts)



Category
Street and Sidewalk Cleaning                             2376640
Graffiti                                                  790427
Parking Enforcement                                       407935
Encampments                                               353885
Abandoned Vehicle                                         271552
General Request - PUBLIC WORKS                            164451
Damaged Property                                          125931
Sewer Issues                                              123462
Tree Maintenance                                          119940
Illegal Postings                                          115694
Litter Receptacles                                         95791
Sign Repair                                                90572
Rec and Park Requests                                      88407
Street Defects                                             88226
Streetlights                                               78219
Sidewalk or Curb

In [217]:
#figure out how to run similar code on different data -> match columns by snyonym and ask user -> are these the same?
#complaint
CSV_PATH = './311_cases.csv'
import subprocess
from tqdm import tqdm
import pandas as pd 
import os
from h3 import h3
import warnings
warnings.filterwarnings('ignore')

def process_date(df_chunk, key):
    df_chunk[key] = df_chunk[key].str.slice(0, 16)
    #df_chunk[key] = pd.to_datetime(df_chunk[key], utc=True, format='%Y-%m-%d %H:%M')

def binRow(df311):
    APERTURE_SIZE = 9
    hex_col = 'hex'+str(APERTURE_SIZE)
    df311[hex_col] = df311.apply(lambda x: h3.geo_to_h3(x.Latitude,x.Longitude,APERTURE_SIZE),1)
    df311g = df311.groupby(hex_col).size()#.to_frame('cnt').reset_index()
    return (df311g)

def saveComplaint(complaint):
    print(complaint)
    data = pdf.loc[pdf['Category'] ==(complaint)]
    binned = binRow(data)
    complaint = complaint.replace(' ', '-').replace('/','-')
    hi = [(hex, count)  for hex,count in binned.to_dict()]
    with open('./data/'+complaint+'.json', 'w') as outfile: json.dump(hi, outfile)
    
stuff = ['Noise - Residential',                         
'HEAT/HOT WATER' ,                              
'Street Condition',                           
'Illegal Parking ',                              
'Blocked Driveway'  ,                            
'Street Light Condition' ,                       
'HEATING'          ,                             
'PLUMBING'     ,                                 
'Water System'  ,                                
'Noise - Street/Sidewalk' ,                      
'GENERAL CONSTRUCTION'  ,                                                                
'UNSANITARY CONDITION']                          
import json
import os
import requests
#get all the airbnb in sf that are not noisy
#getAirbnb() => geocoding mapbox api => lat/lng => h3a9 => lookup in binned table
#address => geocoe => lat/lng => h3a9 => lookup in binned table 

def geoCode(address = "1600 Amphitheatre Parkway, Mountain View, CA"):
    accessToken = "pk.eyJ1IjoiYXdhaGFiIiwiYSI6ImNrdjc3NW11aTJncmIzMXExcXRiNDNxZWYifQ.tqFU7uVd6mbhHtjYsjtvlg"  # Replace with your actual access token

    geocodeUrl = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{address}.json?access_token={accessToken}"

    response = requests.get(geocodeUrl)
    data = response.json()

    if 'features' in data and len(data['features']) > 0:
        location = data['features'][0]['geometry']['coordinates']
        print(f"Longitude: {location[0]}, Latitude: {location[1]}")
        return location
    else:
        
        print("Geocoding failed: No results found")
        
ll = geoCode('450 10th St, San Francisco, CA 94103')
h3Cell = h3.geo_to_h3(ll[0], ll[1], 9)
#figure out how to make fast
#one line to say -> find the least noisy airbnb in SF and NYC
#also fast to say -> find the noisiest airbnb in SF and NYC
#plot both on a map 
#in regular python data analysis -> takes 2-3 hours for an experienced dev
#in english -> takes 16 ms -> onkeypress get instant results
#how to cache 311 -> 3gb for sf and 16gb for nyc -> cant cache on browser
#h3_cells for noise column =
#100ms is probably fine for onKeyPress RPC

#poll = sf, houston, kansas
#get all airbnb in poll.1 if not noisy and near pizza
#AllAirbnbInCityFilteredNoise('sf')
def make_pandas_data_frame():
    print (f'Exact number of rows: {n_rows}')
    df_tmp = pd.read_csv(CSV_PATH, nrows=5)
    df_tmp.head()


    types = {'Category': 'category', 
                  'Longitude': 'float32',
                  'Latitude': 'float32',
            }
    cols = list(types.keys())
    chunksize = 5_000_000

    df_list = [] # list to hold the batch dataframe
    for df_chunk in tqdm(pd.read_csv(CSV_PATH, usecols=cols, dtype=types, chunksize=chunksize)):
        df_list.append(df_chunk) 
    pdf = pd.concat(df_list)
    filtered_df = pdf[(pdf['Latitude'] != 0) & (pdf['Longitude'] != 0)]
    pdf = filtered_df
    del df_list

    APERTURE_SIZE = 9
    hex_col = 'hex'+str(APERTURE_SIZE)
    pdf[hex_col] = 0
    return pdf

def getNoisyCells():
    fp = './data/sf-noise-regions.csv'
    if os.path.isfile(fp):
        print('cached')
        return json.loads(open(fp).read())
    pdf = make_pandas_data_frame()
    data = pdf.loc[pdf['Category'] ==('Noise Report')]
    binned = binRow(data)
    with open(fp, 'w') as f:
        f.write(binned.to_json())
    return binned
getNoisyCells()

def meanDict(_): return sum([_[v] for v in _.keys()]) / len(_)


def isNoisy(nc, apt):
    #latlong = geoCode(addr)
    h3Cell = h3.geo_to_h3(apt['latitude'], apt['longitude'], 9) #.1 square kilometers
    return nc[h3Cell] < meanDict(nc)


def getAirbnbForCity():
    df = pd.read_csv('./data/sf_airbnb.csv')
    airbnb = df[['latitude', 'longitude', 'id', 'listing_url']].to_dict('records')
    return airbnb

def getAllAirbnbInCityThatAreNotNoisy():
    from collections import defaultdict
    all_airbnb = getAirbnbForCity() #make this get for different cities 
    nc = defaultdict(int, getNoisyCells())
    mean = meanDict(nc)
    filtered = [apt for apt in all_airbnb if 
                isNoisy(nc, apt)
                #nc[h3.geo_to_h3(apt['latitude'], apt['longitude'],9)] < mean
               ]
    listing_url = [apt['listing_url'] for apt in filtered]
    return listing_url

one= getAllAirbnbInCityThatAreNotNoisy()
two = airbnb
#one
one

Longitude: -122.411105, Latitude: 37.77049
cached
cached


['https://www.airbnb.com/rooms/3850',
 'https://www.airbnb.com/rooms/12584',
 'https://www.airbnb.com/rooms/24450',
 'https://www.airbnb.com/rooms/24463',
 'https://www.airbnb.com/rooms/25094',
 'https://www.airbnb.com/rooms/25463',
 'https://www.airbnb.com/rooms/29373',
 'https://www.airbnb.com/rooms/29557',
 'https://www.airbnb.com/rooms/32414',
 'https://www.airbnb.com/rooms/39174',
 'https://www.airbnb.com/rooms/39418',
 'https://www.airbnb.com/rooms/41172',
 'https://www.airbnb.com/rooms/41423',
 'https://www.airbnb.com/rooms/42403',
 'https://www.airbnb.com/rooms/42408',
 'https://www.airbnb.com/rooms/42577',
 'https://www.airbnb.com/rooms/47486',
 'https://www.airbnb.com/rooms/47682',
 'https://www.airbnb.com/rooms/51374',
 'https://www.airbnb.com/rooms/51773',
 'https://www.airbnb.com/rooms/53280',
 'https://www.airbnb.com/rooms/54518',
 'https://www.airbnb.com/rooms/54634',
 'https://www.airbnb.com/rooms/55632',
 'https://www.airbnb.com/rooms/55636',
 'https://www.airbnb.com/r

In [212]:
# with open('./asdf.txt', 'w') as f:
#     json.dump(dict(binned), f)
from collections import defaultdict
#json.loads(getNoisyCells())
#meanDict(getNoisyCells())

cached


In [219]:
data = pdf.loc[pdf['Category'] ==('Noise Report')]
binned = binRow(data)
# with open(fp, 'w') as f:
#     json.dump(binned.to_csv(), f)
#dict(binned)

binned

hex9
89283080c83ffff      8
89283080c87ffff     23
89283080c8fffff      1
89283080c93ffff     74
89283080c97ffff     69
89283080c9bffff      2
89283080ca3ffff     27
89283080ca7ffff     14
89283080cafffff      1
89283080cb3ffff     47
89283080cb7ffff     55
89283080cbbffff     30
89283080d93ffff     39
89283080d97ffff      2
89283080d9bffff      6
89283080dd3ffff     11
89283080dd7ffff     90
89283080ddbffff      1
89283080e07ffff      1
89283080e0fffff      4
89283080e13ffff      2
89283080e23ffff      1
89283080e2bffff      9
89283080e3bffff      6
89283080e77ffff      4
89283082003ffff      1
89283082013ffff      1
89283082017ffff      6
89283082027ffff      1
8928308202fffff      1
8928308208bffff      7
8928308208fffff     56
89283082093ffff      1
89283082097ffff      1
892830820a3ffff      5
892830820a7ffff      4
892830820abffff     12
892830820afffff      5
892830820b7ffff     51
892830820bbffff    134
89283082103ffff      1
89283082107ffff      1
8928308210bffff      1
892830

In [122]:
import csv
# with open('./data/sf_airbnb.csv', newline='') as csvfile:
#     #csvreader = csv.(csvfile, delimiter=' ', quotechar='|')
#     for k in csvfile:
#         print(k)
        
        
import pandas as pd

# Load a CSV file into a DataFrame

# Show the DataFrame

df.columns.tolist() #street, city, state, szpcode


df.latitude
df.longitude
df.id



df[['latitude', 'longitude', 'id']]

# Convert the DataFrame to a list of dictionaries
#list_of_dicts = filtered_df.to_dict('records')
#convert df to {latitude: _, longitude: _, listing_id}
#find a list of airbnb that are not noisy -> get a list of links back + a data browser
#df.columns.tolist()

def approxMatch(one, two):
    #get all synonyms for one and two
    return set(one + two)



In [67]:
#saveComplaint('Noise Report')

with open('complaint.csv', 'w') as file:
    file.write(nc.to_csv())